---
title: "Pset V"
author: "Summer Negahdar & Jenny Zhong"
format: 
  pdf:
    keep-tex: true
    include-in-header: 
      text: |
        \usepackage[margin=0.7in]{geometry}
        \usepackage{fvextra}
        \DefineVerbatimEnvironment{Highlighting}{Verbatim}{breaklines,commandchars=\\\{\}}
include-before-body:
  text: |
    \RecustomVerbatimEnvironment{verbatim}{Verbatim}{
      showspaces = false,
      showtabs = false,
      breaksymbolleft={},
      breaklines
    }
execute:
  code-fold: True

---

Partner 1: Summer Negahdar(samarneg)
Partner 2: Jenny Zhong (jzhong1)
This submission is our work alone and complies with the 30538 integrity policy.” Add
your initials to indicate your agreement: **SN** **JZ**
“I have uploaded the names of anyone else other than my partner and I worked with on the problem set here”


In [ ]:
import csv
from bs4 import BeautifulSoup
import pandas as pd
import requests
import altair
import requests
import datetime
from time import sleep
import os
from concurrent.futures import ThreadPoolExecutor

## Develop Initial scraper and crawler

### 1.


In [ ]:
def fetch_page_data(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data from {url}")
        return []  # Return empty list if failed to fetch data

    soup = BeautifulSoup(response.content, 'html.parser')
    data = []
    actions = soup.find_all('li', class_='usa-card card--list pep-card--minimal mobile:grid-col-12')
    for action in actions:
        title_tag = action.find('h2', class_='usa-card__heading')
        title = title_tag.get_text(strip=True) if title_tag else 'No Title Provided'
        
        link_tag = title_tag.find('a')
        link = f"https://oig.hhs.gov{link_tag['href']}" if link_tag else 'No Link Provided'

        date_div = action.find('div', class_='font-body-sm margin-top-1')
        date_tag = date_div.find('span', class_='text-base-dark padding-right-105')
        date = date_tag.get_text(strip=True) if date_tag else 'No Date Provided'
        
        category_ul = action.find('ul', class_='display-inline add-list-reset')
        category_tag = category_ul.find('li')
        category = category_tag.get_text(strip=True) if category_tag else 'No Category Provided'

        data.append({'Title': title, 'Date': date, 'Category': category, 'Link': link})
    return data

# Base URL of the site to scrape
base_url = 'https://oig.hhs.gov/fraud/enforcement/'

# Fetch data from the first page only
first_page_data = fetch_page_data(base_url)


df = pd.DataFrame(first_page_data)

# this scraping is only done on the first page!
print(df.head(5))

### 2.


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_agency_details(link):
    """ Fetches the agency details for a given URL """
    try:
        response = requests.get(link, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            uls = soup.find_all("ul", class_="usa-list usa-list--unstyled margin-y-2")
            for ul in uls:
                spans = ul.find_all("span", class_="padding-right-2 text-base")
                if len(spans) > 1:
                    agency = spans[1]
                    return agency.next_sibling.strip() if agency.next_sibling else 'N/A'
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {link}: {e}")
    return 'N/A'

def process_agencies(df):
    """ Process each link concurrently and fetch agency details """
    links = df['Link'].tolist()
    agencies = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(fetch_agency_details, link): link for link in links}
        for future in as_completed(futures):
            agencies.append(future.result())
    return agencies

# Assuming df is already loaded with the first page data and contains 'Link'
if 'Link' in df:
    df['Agency'] = process_agencies(df)
    print(df.head(5))

# Parse dates and check data types
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
print(df.dtypes)
print(df.head())

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

# Check the data type to confirm
print(df.dtypes)
print(df.head())
print(len(df))

## Making the scraper dynamic

### 1.

a. 
Step 1: Set up function and input validation 
Function: dynamic_scraper(month, year), month and year are parameters. Check if year is at least 2013; if not, print a warning and exit function. 

Step 2: Initialize URL and variables
Set the base URL for the enforcement actions page. Initialize an empty list to store the data and create a datetime object, base_date_obj, to represent the starting date using month and year. Initialize end_flag as True to contol page looping. 

Step 3: Loop through pages until end condition
Use a while loop that continues as long as end_flag is True. For each page, retrieve the HTML content and parse it using BeautifulSoup to locate the list of enforcement actions. 

Step 4: Extract data from each enforcement action
For each action on the page, extract: title, data (converted to datetime for comparison), category and link. If the date is older than base_date_obj, set end_flag to False to stop the loop. For each link, visit the details page to retrieve the agency, name. 

Step 5: Pause and continue to the next page
Add a 1-second delay between pages to prevent server-side blockng. If a "next page" link exists, update the URL and continue, otherwise exit the loop. 

Step 6: Save data to CSV and print preview 
Convert the collected data to a pandas DataFrame. Save the DataFrame to a CSV file named enforcement_actions_year_month.csv and print the first few rows for verification. 

b. Based on the output, the total number of enforcement actions collected is 1534. 
Total enforcement actions collected: 1534
Earliest enforcement action scraped:
Date: 2023-01-03 00:00:00
Title: Podiatrist Pays $90,000 To Settle False Billing Allegations
Category: Criminal and Civil Actions
Agency: January 3, 2023
Link: https://oig.hhs.gov/fraud/enforcement/podiatrist-pays-90000-to-settle-false-billing-allegations/


In [ ]:
from time import sleep
from datetime import datetime

def scrape_all_pages(base_url, month, year, output_file):
    page_number = 1
    base_date_obj = datetime(year, month, 1) 
    stop_scraping = False

    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Title", "Date", "Category", "Link"])
        writer.writeheader()  # Write the header row once

        while not stop_scraping:
            url = f"{base_url}?page={page_number}"
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            actions = soup.find_all('li', class_='usa-card card--list pep-card--minimal mobile:grid-col-12')
            
            if not actions:
                print(f"No more actions found on page {page_number}.")
                break

            for action in actions:
                title = action.find('h2', class_='usa-card__heading').get_text(strip=True)
                link = "https://oig.hhs.gov" + action.find('h2').find('a')['href']
                date_text = action.find('span', class_='text-base-dark padding-right-105').get_text(strip=True)
                date_object = datetime.strptime(date_text, "%B %d, %Y")
                category = action.find('ul', class_="display-inline add-list-reset").get_text(strip=True)
                
                if date_object < base_date_obj:
                    stop_scraping = True
                    break
                
                writer.writerow({'Title': title, 'Date': date_text, 'Category': category, 'Link': link})
            
            page_number += 1
            sleep(1)  
        
def fetch_agency(link):
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')
        agency_tag = soup.find("ul", class_="usa-list usa-list--unstyled margin-y-2")
        if agency_tag:
            agency = agency_tag.find("span", class_="padding-right-2 text-base")
            return agency.next_sibling.strip() if agency and agency.next_sibling else 'N/A'
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {link}: {e}")
    return 'N/A'

def add_agencies_to_csv(output_file):
    df = pd.read_csv(output_file)
    df['Agency'] = get_agencies(df)
    df.to_csv(output_file, index=False)

def get_agencies(df):
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_index = {executor.submit(fetch_agency, row['Link']): index for index, row in df.iterrows()}
        agencies = ['N/A'] * len(df)
        for future in as_completed(future_to_index):
            index = future_to_index[future]
            agencies[index] = future.result()
    return agencies

base_url = 'https://oig.hhs.gov/fraud/enforcement/'
output_file = "enforcement_actions_since_jan2023.csv"

scrape_all_pages(base_url, 1, 2023, output_file)

add_agencies_to_csv(output_file)

print("Data scraped and saved to enforcement_actions_since_jan2023.csv")

Checking the number of rows and the details. 

In [ ]:
output_file = "enforcement_actions_since_jan2023.csv"
final_df = pd.read_csv(output_file)

num_enforcement_actions = len(df)
print(f"Total enforcement actions collected: {num_enforcement_actions}")

final_df['Date'] = pd.to_datetime(final_df['Date'], errors='coerce')

earliest_action = final_df.sort_values(by='Date').iloc[0]

print("\nEarliest enforcement action scraped:")
print(f"Date: {earliest_action['Date']}")
print(f"Title: {earliest_action['Title']}")
print(f"Category: {earliest_action['Category']}")
print(f"Agency: {earliest_action['Agency']}")
print(f"Link: {earliest_action['Link']}")

c. 


In [ ]:
df = scrape_all_pages(base_url, 1, 2021)

# The rest remains the same as above, starting from converting the 'Date' and filtering
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df_2021 = df[df['Date'] >= datetime(2021, 1, 1)]
print(df_2021.head(5))

## Plot data based on scraped data (using Altair)

### 1.



### 2.


In [ ]:
##manually categorizing
def categorize_topic(title):
    if 'health' in title.lower():
        return 'Health Care'
    elif any(word in title.lower() for word in ['financial', 'bank']):
        return 'Financial Fraud'
    elif 'drug' in title.lower():
        return 'Drug Enforcement'
    elif any(word in title.lower() for word in ['bribery', 'corruption']):
        return 'Bribery/Corruption'
    else:
        return 'Other'


df_2021['Topic'] = df_2021['Title'].apply(categorize_topic)

df_2021['Month'] = df_2021['Date'].dt.to_period('M')

# Group by Month and Topic, and count entries
summary = df_2021.groupby(['Month', 'Topic']).size().reset_index(name='Counts')

# Ensure the 'Month' is treated as a Date type for plotting
summary['Month'] = summary['Month'].dt.to_timestamp()

# Plotting with Altair
chart = alt.Chart(summary).mark_line().encode(
    x='Month:T',  
    y='Counts:Q',
    color='Topic:N',
    tooltip=['Month', 'Counts', 'Topic']
).properties(
    width=600,
    height=300,
    title='Enforcement Actions Over Time by Topic'
)
chart.display()

## Create maps of enforcement activity


### 1.


In [ ]:
#I need to extract state names from agency column:
df_2021['State'] = df_2021['Agency'].apply(lambda x: x.split("State of ")[-1] if "District of " in x else None)
print(df_2021.head(5))

### 2.




## Extra Credit: Calculate the enforcement actions on a per-capita basis

### 1.



### 2.



### 3.
